In [1]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from core import semcor_bert_pipeline
from core.metrics import *

In [2]:
shared_metrics = {}
shared_words = ['foot.n', 'plane.n', 'model.n', 'degree.n']
fmt_sense_12 = lambda w: [w + '.01', w + '.02']
shared_senses = [fmt_sense_12(w) for w in shared_words]
shared_senses[-1] = ['degree.n.01', 'academic_degree.n.01']
shared_senses[1] = ['airplane.n.01', 'plane.n.02']
for w_s in zip(shared_words, shared_senses):
    model_data = binary_logistic(w_s[0], w_s[1])
    weight_values, weight_indices = nonzero_weights(model_data['model'])
    f_scores, accuracies, wrong_indices = k_fold_cv(model_data['data'], model_data['transformed_labels'])
    shared_metrics[w_s[0]] = {'senses': w_s[1], 'data': model_data, 'weights': weight_values,
                    'weight_indices': weight_indices, 'f1_kfold': f_scores, 'acc_kfold': accuracies,
                             'incorrect_indices': wrong_indices}
    

/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metr

In [3]:
shared_stats = []
incorrect = []
for k in shared_metrics:
    word_results = shared_metrics[k]
    weights = word_results['weights']
    incorrect.append(misclassified_sentences(word_results['data'], word_results['incorrect_indices']))
    shared_stats.append({'word_type': k, 'avg_f1': np.mean(word_results['f1_kfold']), 'avg_acc': np.mean(word_results['acc_kfold']),
    'pct_nonzero_weights': len(weights) / 768, 'max_wt': max(weights), 'min_wt': min(weights), 'mean_nonzero_wt': np.mean(weights), 'sd_nonzero_wt': np.std(weights)})
pd.DataFrame(shared_stats)

,word_type,avg_f1,avg_acc,pct_nonzero_weights,max_wt,min_wt,mean_nonzero_wt,sd_nonzero_wt
0,foot.n,0.600000,0.973333,0.031250,0.221690,-0.230311,0.029156,0.132311
1,plane.n,0.333333,0.940000,0.015625,0.499859,-0.255368,0.056093,0.224953
2,model.n,0.291429,0.874242,0.036458,0.258451,-0.274350,0.019918,0.137800
3,degree.n,0.800000,1.000000,0.018229,0.456464,-0.226661,0.001347,0.166190


Misclassified senses (Only binary classification, 3/16 senses for math plane were misclassified)

In [4]:
pd.set_option('display.max_colwidth', 500)
pd.concat(incorrect)

,true_label,sentences
0,foot.n.02,"When it comes to rate of early growth , the Indian python leads with a figure of about 3 feet 6 inches per year for the first two years , more or less ."
1,foot.n.02,"He reasons that as anacondas 30 feet long are often found , some might be 38 , and occasional `` monstrous freaks '' over 50 ."
2,foot.n.02,He rejects dimensions of 70 feet and more .
3,foot.n.02,"Hans Schweizer had one that increased from 19 - 1 2 inches to 5 feet 3 inches in five years , and J . J . Quelch records a growth of from less than 4 feet to nearly 10 in about six years ."
0,plane.n.02,"If a is the major axis of an ellipsoid and b and c are the other two axes , the radius of curvature in the ab plane at the end of the axis is * * f , and the difference in pressure along the a and b axes is * * f ."
1,plane.n.02,"Also , planetary gravitational attraction increases the dust concentration near the plane of the ecliptic as the sun is approached ."
2,plane.n.02,We will refer to the plane of C and **f as the C-plane and to the plane of the graph as the f-plane .
0,model.n.01,The Glazer-Fine Arts edition ( Concert-Disc ) is a model of lucidity and organization .
1,model.n.01,"I asked Quasimodo recently how he accomplished this , and he replied that he had painted his model `` a beautiful shade of red and then had her breathe on the canvas '' , which was his typical tongue-in-cheek way of chiding me for my lack of sensitivity ."
2,model.n.01,The Glazer-Fine Arts edition ( Concert-Disc ) is a model of lucidity and organization .


I thought this might be useful to see if weights at similar positions were used.

In [212]:
[(k, shared_metrics[k]['weight_indices']) for k in shared_words]

[('foot.n',
  array([ 13,  29,  70,  89, 141, 158, 191, 226, 231, 287, 304, 308, 493,
         518, 532, 547, 637, 664, 693, 709, 730])),
 ('table.n',
  array([  9,  37,  49, 136, 143, 286, 308, 317, 332, 334, 393, 398, 445,
         471, 609, 619, 666, 680])),
 ('plane.n',
  array([105, 157, 254, 286, 308, 409, 411, 414, 448, 480, 513, 619, 637,
         685, 695])),
 ('right.n', array([ 58,  62,  65, 135, 184, 308, 338, 514, 606, 747])),
 ('model.n',
  array([ 74, 121, 161, 282, 308, 340, 364, 423, 434, 450, 452, 473, 525,
         552, 586, 619, 739])),
 ('degree.n',
  array([ 15,  22, 220, 262, 328, 350, 432, 523, 541, 544, 565, 620, 724]))]